# Pacing Equilibrium in First Price Auction Markets (Part V)

In [1]:
import numpy as np

In [2]:
def get_bidder_demand(prices, values, budget, min_roas = 1.0):
    """
        prices: [nitems, 1]
        utilities: [nitems, 1]
        budget: float
    """
    nitems = len(prices)
    demand = np.zeros_like(prices)
    utility = 0
    cost = 0
    
    # sort items in descending order of bang-per-buck 
    idx = sorted(range(nitems), key=lambda i: values[i] / prices[i], reverse=True)
    
    # demand items in that order until the budget constraint or ROAS constraint is reached
    i = 0
    while i < nitems and cost <= budget:
        ix = idx[i]
        
        if values[ix] / prices[ix] >= min_roas:
            demand[ix] = (budget - cost) / prices[ix]
        else:
            demand[ix] = 0
        
        # update utility and cost after getting the item
        utility += demand[ix] * values[ix]
        cost += demand[ix] * prices[ix]
        i += 1
    
    return demand


def fppe_tatonnement(budgets, values, min_roas=None, niter=100, eps=0.01, start_price=1.0, nprint=1):
    """
        budgets: [nbidders, 1]
        utilities: [nbidders, nitems]
        min_roas: [nbidders, 1]
    """
    
    nbidders, nitems = values.shape
    assert nbidders == budgets.shape[0]
    
    prices = np.full((nitems,), fill_value=start_price)
    if min_roas is None:
        min_roas = np.ones((nbidders, 1))
    else:
        assert nbidders == min_roas.shape[0]
    
    sum_sigm = 0
    sum_total_demand = np.zeros_like(prices)
    sum_prices = np.zeros_like(prices)
    sum_bidder_demand = np.zeros((nbidders, nitems))
    
    
    print("iter\tp1\tp2\td1_1\td1_2\td2_1\td2_2\tv1\tv2\ts1\ts2\tbeta1\tbeta2")
    
    for it in range(niter):
        primal_value = 0.0
        dual_value = prices.sum()
        bidder_demand = np.zeros((nbidders, nitems))
        total_demand = np.zeros_like(prices)
        
        for j in range(nbidders):
            bidder_demand[j, :] = get_bidder_demand(prices, values[j, :], budgets[j], min_roas[j])
            total_demand += bidder_demand[j, :]
        
        sigm = 1. / np.max(total_demand)
        sum_bidder_demand += sigm * bidder_demand
        sum_total_demand += sigm * total_demand
        sum_prices += sigm * prices
        sum_sigm += sigm
        
        prices = prices * (1 + eps * sigm * (total_demand - np.ones_like(prices)))
        
        if it % nprint == 0 or it == (niter - 1):
            avg_total_demand = sum_total_demand / sum_sigm
            avg_bidder_demand = sum_bidder_demand / sum_sigm
            avg_prices = sum_prices / sum_sigm
            avg_value = np.sum(avg_bidder_demand * values, axis=1)
            avg_spend = np.sum(avg_bidder_demand * avg_prices, axis=1)
            avg_beta = budgets / (avg_value + (budgets - avg_spend))
            print(
                "%d\t%6.4f\t%6.4f\t%6.4f\t%6.4f\t%6.4f\t%6.4f\t%6.4f\t%6.4f\t%6.4f\t%6.4f\t%6.4f\t%6.4f" % (
                    it, 
                    avg_prices[0],           avg_prices[1], 
                    avg_bidder_demand[0, 0], avg_bidder_demand[0, 1],
                    avg_bidder_demand[1, 0], avg_bidder_demand[1, 1],
                    avg_value[0],            avg_value[1],
                    avg_spend[0],            avg_spend[1],
                    avg_beta[0],             avg_beta[1]
                )
            )
                   
 
    return avg_prices    

# Example 1

In [3]:
budgets = np.array([5.0, 2.0])
values = np.array([
    [2.0, 1.0],
    [3.0, 2.0]
])

prices = fppe_tatonnement(budgets, values, niter=10000, eps=0.01, nprint=500)

iter	p1	p2	d1_1	d1_2	d2_1	d2_2	v1	v2	s1	s2	beta1	beta2
0	1.0000	1.0000	5.0000	0.0000	2.0000	0.0000	10.0000	6.0000	5.0000	2.0000	0.5000	0.3333
500	1.9550	1.2861	1.0021	0.0000	0.3063	1.1258	2.0042	3.1706	1.9591	2.0468	0.9911	0.6402
1000	1.9800	1.3126	0.8316	0.0000	0.3192	1.0618	1.6632	3.0814	1.6466	2.0260	0.9967	0.6546
1500	1.9880	1.3210	0.7762	0.0000	0.3240	1.0407	1.5525	3.0533	1.5432	2.0188	0.9981	0.6591
2000	1.9918	1.3251	0.7505	0.0000	0.3258	1.0311	1.5010	3.0396	1.4949	2.0152	0.9988	0.6613
2500	1.9941	1.3271	0.7332	0.0000	0.3280	1.0241	1.4664	3.0321	1.4621	2.0131	0.9991	0.6625
3000	1.9957	1.3284	0.7227	0.0000	0.3285	1.0208	1.4455	3.0271	1.4424	2.0116	0.9994	0.6632
3500	1.9968	1.3295	0.7151	0.0000	0.3294	1.0176	1.4302	3.0233	1.4279	2.0106	0.9995	0.6638
4000	1.9976	1.3300	0.7102	0.0000	0.3295	1.0162	1.4203	3.0210	1.4186	2.0098	0.9997	0.6642
4500	1.9982	1.3308	0.7057	0.0000	0.3299	1.0144	1.4114	3.0186	1.4101	2.0092	0.9997	0.6646
5000	1.9987	1.3309	0.7022	0.0000	0.3306	1.0129	1.4043	3.01

# Example 2

In [4]:
budgets = np.array([5.0, 2.0])
values = np.array([
    [2.0, 1.0],
    [3.0, 2.0]
])
min_roas = np.array([2.0, 2.0])

prices = fppe_tatonnement(budgets, values, min_roas, niter=10000, eps=0.01, nprint=500)

iter	p1	p2	d1_1	d1_2	d2_1	d2_2	v1	v2	s1	s2	beta1	beta2
0	1.0000	1.0000	5.0000	0.0000	2.0000	0.0000	10.0000	6.0000	5.0000	2.0000	0.5000	0.3333
500	1.1935	0.8091	0.0028	0.0000	1.0727	0.9129	0.0057	5.0440	0.0034	2.0189	0.9995	0.3980
1000	1.1961	0.8053	0.0014	0.0000	1.0361	0.9597	0.0029	5.0278	0.0017	2.0122	0.9998	0.3988
1500	1.1975	0.8034	0.0010	0.0000	1.0249	0.9739	0.0019	5.0226	0.0011	2.0098	0.9998	0.3990
2000	1.1978	0.8029	0.0007	0.0000	1.0193	0.9810	0.0014	5.0199	0.0009	2.0086	0.9999	0.3991
2500	1.1983	0.8022	0.0006	0.0000	1.0159	0.9853	0.0011	5.0184	0.0007	2.0079	0.9999	0.3992
3000	1.1984	0.8021	0.0005	0.0000	1.0131	0.9889	0.0010	5.0173	0.0006	2.0074	0.9999	0.3992
3500	1.1987	0.8017	0.0004	0.0000	1.0116	0.9909	0.0008	5.0166	0.0005	2.0070	0.9999	0.3992
4000	1.1987	0.8017	0.0004	0.0000	1.0105	0.9923	0.0007	5.0160	0.0004	2.0067	0.9999	0.3993
4500	1.1989	0.8015	0.0003	0.0000	1.0096	0.9934	0.0006	5.0155	0.0004	2.0065	0.9999	0.3993
5000	1.1988	0.8015	0.0003	0.0000	1.0085	0.9948	0.0006	5.01

# Shaded budget

In [5]:
budgets = np.array([5.0, 1.69])
values = np.array([
    [2.0, 1.0],
    [3.0, 2.0]
])
min_roas = np.array([2.0, 2.0])

prices = fppe_tatonnement(budgets, values, min_roas, niter=10000, eps=0.01, nprint=500)

iter	p1	p2	d1_1	d1_2	d2_1	d2_2	v1	v2	s1	s2	beta1	beta2
0	1.0000	1.0000	5.0000	0.0000	1.6900	0.0000	10.0000	5.0700	5.0000	1.6900	0.5000	0.3333
500	1.0717	0.7289	0.0028	0.0000	1.0101	0.8597	0.0056	4.7496	0.0030	1.7091	0.9995	0.3573
1000	1.0454	0.7055	0.0014	0.0000	1.0026	0.9287	0.0029	4.8652	0.0015	1.7034	0.9997	0.3483
1500	1.0355	0.6958	0.0010	0.0000	1.0024	0.9525	0.0019	4.9123	0.0010	1.7008	0.9998	0.3448
2000	1.0299	0.6913	0.0007	0.0000	1.0024	0.9648	0.0015	4.9367	0.0008	1.6993	0.9999	0.3430
2500	1.0269	0.6882	0.0006	0.0000	1.0024	0.9722	0.0012	4.9515	0.0006	1.6984	0.9999	0.3419
3000	1.0245	0.6864	0.0005	0.0000	1.0019	0.9779	0.0010	4.9615	0.0005	1.6977	0.9999	0.3412
3500	1.0231	0.6849	0.0004	0.0000	1.0019	0.9814	0.0008	4.9686	0.0004	1.6972	0.9999	0.3406
4000	1.0218	0.6839	0.0004	0.0000	1.0020	0.9840	0.0007	4.9740	0.0004	1.6969	0.9999	0.3402
4500	1.0210	0.6830	0.0003	0.0000	1.0020	0.9860	0.0007	4.9782	0.0003	1.6966	0.9999	0.3399
5000	1.0202	0.6824	0.0003	0.0000	1.0018	0.9881	0.0006	4.98